----
<img src="../../files/lseg.svg" class="rft-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Access layer - Pricing streams
This notebook demonstrates how to use the pricing stream of the Access layer of the LSEG Data Library to retrieve streaming level 1 pricing data. The Access layer provides a simplified pricing streams interface designed for FinCoders. The complete pricing stream interface is available in the Content layer of the library.

#### About the open_pricing_stream function 
The **open_pricing_stream()** function allows you create and open a pricing stream object to retrieve streaming data via a single function call.


***open_pricing_stream()*** parameters: 
- ***universe***: Single instrument or list of instruments to subscribe to
    - Type: String or list
    - Optional: No  
- ***fields***: Single field or list of requested fields
    - Type: String or list
    - Optional: Yes  
- ***service***: Name of the streaming service publishing the instruments
    - Type: String
    - Optional: Yes  
- ***on_data***: Callback function to receive incoming data events
    - Type: Function
    - Optional: Yes  

**open_pricing_stream()** returns a PricingStream object.

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Data Library Q&A Forum](https://community.developers.refinitiv.com/smart-spaces/521/index.html). 
The LSEG Developer Community will be happy to help. 

----

## Some Imports to start with

In [15]:
import lseg.data as ld
import datetime
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [16]:
ld.open_session()

<lseg.data.session.Definition object at 0x11e8d9df0 {name='workspace'}>

## Retrieve data

### Using pricing streams with events

#### Define a callback to receive data events

In [17]:
def display_data(data, instrument, stream):
    clear_output(wait=True)
    current_time = datetime.datetime.now().time()
    print(current_time, "- Data received for", instrument)
    display(data)

#### Open the stream and register the callback

In [18]:
stream = ld.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK'],
    on_data=display_data
)

09:21:00.186412 - Data received for GBP=


,ASK,BID
GBP=,1.3178,1.3171


#### Close the stream

In [19]:
stream.close()

<OpenState.Closed: 'Closed'>

### Using pricing streams as a data cache

#### Create and open a Pricing stream object

The Pricing stream object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, the Pricing stream will retrieve all fields available for the requested instruments

In [20]:
stream = ld.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK']
)

As soon as the method returns, the stream object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from LSEG Workspace or the LSEG Data Platform. All this happens behind the scene, waiting for your application to pull out data from the cache.  

#### Extract snapshot data from the streaming cache
Once the stream is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot can be called any number of times. As these calls return the latest received values, successive calls to get_snapshot may return different values. Returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh values. 

In [21]:
stream.get_snapshot()

,Instrument,ASK,BID
0,GBP=,1.3174,1.3173
1,EUR=,1.1114,1.1113
2,JPY=,142.69,142.68


#### Get a snapshot for a subset of instruments and fields

In [22]:
stream.get_snapshot(
    universe = ['EUR=', 'GBP='], 
    fields = ['BID', 'ASK']
)

,Instrument,BID,ASK
0,EUR=,1.1113,1.1114
1,GBP=,1.3173,1.3174


#### Other options to get values from the streaming cache

##### Direct access to real-time fields

In [23]:
print('GBP/BID:', stream['GBP=']['BID'])
print('EUR/BID:', stream['EUR=']['BID'])

GBP/BID: 1.3171
EUR/BID: 1.1113


##### Direct acces to a streaming instrument

In [24]:
gbp = stream['GBP=']
print(gbp['BID'])

1.3171


##### Iterate on fields

In [25]:
print('GBP=')
for field_name, field_value in stream['GBP=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in stream['JPY=']:
    print('\t' + field_name + ': ', field_value)

GBP=
	ASK:  1.3176
	BID:  1.3171
JPY=
	ASK:  142.71
	BID:  142.66


##### Iterate on streaming instruments and fields

In [26]:
for streaming_instrument in stream:
    print(streaming_instrument.name)
    for field_name, field_value in streaming_instrument:
        print('\t' + field_name + ': ', field_value)

GBP=
	ASK:  1.3176
	BID:  1.3171
EUR=
	ASK:  1.1114
	BID:  1.1113
JPY=
	ASK:  142.69
	BID:  142.68


#### Close the stream

In [27]:
stream.close()

<OpenState.Closed: 'Closed'>

### Close the session

In [28]:
ld.close_session()